In [36]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import json
from influxdb_client import InfluxDBClient
from influxdb_client.client.write_api import SYNCHRONOUS
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error
import math

In [2]:
fd =  open('../../server/config.json', 'r')
config = json.load(fd)
query_temp = 'from(bucket:"temp")' \
        ' |> range(start: -3h)'\
        ' |> filter(fn: (r) => r._measurement == "val" and r._field == "value")'
client =  InfluxDBClient(url='http://'+config["influx"]["remotehost"]+':'+config["influx"]["port"], token=config["influx"]["token"], org=config["influx"]["org"], debug=False)
write_api = client.write_api(write_options=SYNCHRONOUS)
query_api = client.query_api()  
result = query_api.query_data_frame(query_temp)
df = result.drop(columns=['result', 'table', '_start', '_stop', '_field', '_measurement'])
df_temp = df.copy()
df_temp = df_temp.loc[df_temp['sensor'] == '0']
df_temp = df_temp.drop(columns=['lat', 'lon', 'sensor'])
df_temp = df_temp.rename(columns = {'_time': 'Time', '_value': 'y'})
df_temp['Time'] = df_temp['Time'].dt.strftime('%d-%m-%Y %H:%M:%S')
df_temp
fd.close()

In [84]:
l = len(df_temp.values)
rmse_temp = []
n_predicted_temp = []
for i in range(1, 101, 20):
    splitsub = int(l*(i/100))
    subset = df_temp['y'][:splitsub]
    splitpoint = int(splitsub*0.80)
    train = subset[:splitpoint]
    test = subset[splitpoint:]
    model = ARIMA(train, order=(1,1,1)) 
    model_fit = model.fit()
    n_predicted_temp.append(len(test))
    predictions = model_fit.forecast(len(test))
    rmse_temp.append(math.sqrt(mean_squared_error(test, predictions)))

C:\Users\franc\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [85]:
df_rmse_temp = pd.DataFrame(data=rmse_temp, index=None)
print(df_rmse_temp.describe().transpose(), n_predicted_temp)


      count      mean       std  min       25%       50%       75%       max
ping    5.0  0.123649  0.134416  0.0  0.069693  0.073725  0.124073  0.350756 [2, 46, 89, 132, 175]


In [86]:
forecast_temp = model_fit.get_forecast()
forecast_temp.summary_frame()
mean_temp = forecast_temp.predicted_mean
conf_int_temp = forecast_temp.conf_int(alpha=0.05)
print('Temperature Model Evaluation Summary:')
print('Mean: ', mean_temp)
print('Confidence Interval: ', conf_int_temp)

Temperature Model Evaluation Summary:
Mean:  697    28.9
dtype: float64
Confidence Interval:         lower y    upper y
697  28.830257  28.969742


In [75]:
fd =  open('../../server/config.json', 'r')
config = json.load(fd)
query_temp = 'from(bucket:"hum")' \
        ' |> range(start: -3h)'\
        ' |> filter(fn: (r) => r._measurement == "val" and r._field == "value")'
client =  InfluxDBClient(url='http://'+config["influx"]["remotehost"]+':'+config["influx"]["port"], token=config["influx"]["token"], org=config["influx"]["org"], debug=False)
write_api = client.write_api(write_options=SYNCHRONOUS)
query_api = client.query_api()  
result = query_api.query_data_frame(query_temp)
df = result.drop(columns=['result', 'table', '_start', '_stop', '_field', '_measurement'])
df_hum = df.copy()
df_hum = df_hum.loc[df_hum['sensor'] == '0']
df_hum = df_hum.drop(columns=['lat', 'lon', 'sensor'])
df_hum = df_hum.rename(columns = {'_time': 'Time', '_value': 'y'})
df_hum['Time'] = df_hum['Time'].dt.strftime('%d-%m-%Y %H:%M:%S')
df_hum
fd.close()

In [81]:
l = len(df_hum.values)
rmse_hum = []
n_predicted_hum = []
for i in range(1, 101, 20):
    splitsub = int(l*(i/100))
    subset = df_hum['y'][:splitsub]
    splitpoint = int(splitsub*0.80)
    train = subset[:splitpoint]
    test = subset[splitpoint:]
    model = ARIMA(train, order=(1,1,1)) 
    model_fit = model.fit()
    n_predicted_hum.append(len(test))
    predictions = model_fit.forecast(len(test))
    rmse_hum.append(math.sqrt(mean_squared_error(test, predictions)))

In [82]:
df_rmse_hum = pd.DataFrame(data=rmse_hum, index=None)
print(df_rmse_hum.describe().transpose(), n_predicted_hum)

      count     mean       std       min       25%       50%       75%  \
ping    5.0  0.22587  0.241021  0.088111  0.096149  0.122514  0.169299   

           max  
ping  0.653277   [3, 51, 100, 149, 198]


In [83]:
forecast_hum = model_fit.get_forecast()
forecast_hum.summary_frame()
mean_hum = forecast_hum.predicted_mean
conf_int_hum = forecast_hum.conf_int(alpha=0.05)
print('Temperature Model Evaluation Summary:')
print('Mean: ', mean_hum)
print('Confidence Interval: ', conf_int_hum)

Temperature Model Evaluation Summary:
Mean:  788    45.702824
dtype: float64
Confidence Interval:         lower y    upper y
788  45.503475  45.902173


In [72]:
fd =  open('../../server/config.json', 'r')
config = json.load(fd)
query_temp = 'from(bucket:"smoke")' \
        ' |> range(start: -3h)'\
        ' |> filter(fn: (r) => r._measurement == "val" and r._field == "value")'
client =  InfluxDBClient(url='http://'+config["influx"]["remotehost"]+':'+config["influx"]["port"], token=config["influx"]["token"], org=config["influx"]["org"], debug=False)
write_api = client.write_api(write_options=SYNCHRONOUS)
query_api = client.query_api()  
result = query_api.query_data_frame(query_temp)
df = result.drop(columns=['result', 'table', '_start', '_stop', '_field', '_measurement'])
df_gas = df.copy()
df_gas = df_gas.loc[df_gas['sensor'] == '0']
df_gas = df_gas.drop(columns=['lat', 'lon', 'sensor'])
df_gas = df_gas.rename(columns = {'_time': 'Time', '_value': 'y'})
df_gas['Time'] = df_gas['Time'].dt.strftime('%d-%m-%Y %H:%M:%S')
df_gas
fd.close()

In [78]:
l = len(df_gas.values)
rmse_gas = []
n_predicted_gas = []
for i in range(1, 101, 20):
    splitsub = int(l*(i/100))
    subset = df_gas['y'][:splitsub]
    splitpoint = int(splitsub*0.80)
    train = subset[:splitpoint]
    test = subset[splitpoint:]
    model = ARIMA(train, order=(1,1,1)) 
    model_fit = model.fit()
    n_predicted_gas.append(len(test))
    predictions = model_fit.forecast(len(test))
    rmse_gas.append(math.sqrt(mean_squared_error(test, predictions)))

In [79]:
df_rmse_gas = pd.DataFrame(data=rmse_gas, index=None)
print(df_rmse_gas.describe().transpose(), n_predicted_gas)


      count      mean       std       min       25%       50%       75%  \
ping    5.0  0.085217  0.081899  0.025416  0.051669  0.052194  0.067616   

           max  
ping  0.229189   [3, 52, 100, 149, 198]


In [80]:
forecast_gas = model_fit.get_forecast()
forecast_gas.summary_frame()
mean_gas = forecast_gas.predicted_mean
conf_int_gas = forecast_gas.conf_int(alpha=0.05)
print('Temperature Model Evaluation Summary:')
print('Mean: ', mean_gas)
print('Confidence Interval: ', conf_int_gas)

Temperature Model Evaluation Summary:
Mean:  790    2.827376
dtype: float64
Confidence Interval:        lower y  upper y
790  2.734632  2.92012
